# Analyse et nettoyage du jeu de données 

In [2]:
import pandas as pd

### Chargement du fichier excel et transformation de chaque feuille en csv

In [3]:
# # Chemin vers le fichier Excel d'entrée
# excel_file_path = 'online_retail_II.xlsx'

# # Charger le fichier Excel en utilisant pandas
# xls = pd.ExcelFile(excel_file_path)

# # Parcourir chaque feuille de calcul du fichier Excel
# for sheet_name in xls.sheet_names:
#     # Charger la feuille de calcul en tant que dataframe
#     data_frame = xls.parse(sheet_name)
    
#     # Chemin vers le fichier CSV de sortie
#     csv_file_path = f'online_retail{sheet_name}.csv'
    
#     # Convertir le dataframe en fichier CSV
#     data_frame.to_csv(csv_file_path, index=False, encoding='utf-8')
    
# print('Conversion terminée.')


### Concaténation des deux feuilles en un fichier csv

In [4]:
# # Chemin vers le premier fichier CSV
# csv_file1 = 'online_retailYear 2009-2010.csv'

# # Chemin vers le deuxième fichier CSV
# csv_file2 = 'online_retailYear 2010-2011.csv'

# # Charger les deux fichiers CSV en utilisant pandas
# data_frame1 = pd.read_csv(csv_file1)
# data_frame2 = pd.read_csv(csv_file2)

# # Concaténer les deux dataframes
# concatenated_data = pd.concat([data_frame1, data_frame2])

# # Chemin vers le fichier CSV de sortie
# output_csv = 'online_retail.csv'

# # Enregistrer le dataframe concaténé en tant que fichier CSV
# concatenated_data.to_csv(output_csv, index=False)

# print('Concaténation terminée.')


## Nettoyage des données

In [5]:
df_ukretail = pd.read_csv('online_retail.csv')


In [6]:
df_ukretail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067371 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   Invoice      1067371 non-null  object 
 1   StockCode    1067371 non-null  object 
 2   Description  1062989 non-null  object 
 3   Quantity     1067371 non-null  int64  
 4   InvoiceDate  1067371 non-null  object 
 5   Price        1067371 non-null  float64
 6   Customer ID  824364 non-null   float64
 7   Country      1067371 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 65.1+ MB


In [7]:
df_ukretail.shape

(1067371, 8)

In [8]:
df_ukretail.head(10)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom
5,489434,22064,PINK DOUGHNUT TRINKET POT,24,2009-12-01 07:45:00,1.65,13085.0,United Kingdom
6,489434,21871,SAVE THE PLANET MUG,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom
7,489434,21523,FANCY FONT HOME SWEET HOME DOORMAT,10,2009-12-01 07:45:00,5.95,13085.0,United Kingdom
8,489435,22350,CAT BOWL,12,2009-12-01 07:46:00,2.55,13085.0,United Kingdom
9,489435,22349,"DOG BOWL , CHASING BALL DESIGN",12,2009-12-01 07:46:00,3.75,13085.0,United Kingdom


### Identifier les valeurs manquantes

In [9]:
nan_count = df_ukretail.isna().sum()
print(nan_count.sort_values)

<bound method Series.sort_values of Invoice             0
StockCode           0
Description      4382
Quantity            0
InvoiceDate         0
Price               0
Customer ID    243007
Country             0
dtype: int64>


Garde-t-on les lignes où le client n'est pas identifié ? Sachant qu'on cherche à classifier les clients. Nous faisons le choix de les supprimer.

In [10]:
df_ukretail = df_ukretail.dropna(subset=['Customer ID'])

### Identifier les doublons

Il existe des lignes identiques, mais on ne sait pas si ce sont réellement des doublons ou si cela peut s'expliquer par d'autres informations dont on ne dispose pas. Nous choisissons donc de garder les lignes.

### Conversion des types

In [11]:
df_ukretail['Customer ID'] = df_ukretail['Customer ID'].astype('str')

### Consistance des données

In [12]:
df_ukretail.describe()

,Quantity,Price
count,824364.000000,824364.000000
mean,12.414574,3.676800
std,188.976099,70.241388
min,-80995.000000,0.000000
25%,2.000000,1.250000
50%,5.000000,1.950000
75%,12.000000,3.750000
max,80995.000000,38970.000000


Les lignes où les quantités sont négatives correspondent à des commandes annulées. Il faut les garder pour que les dépenses des clients soient réalistes et s'équilibrent entre la commande puis son annulation.

Les commandes où le prix est à 0 correspond probablement à des cadeaux. Nous choisissons de supprimer les lignes, qui ne nous apprendrons rien sur le comportement des clients.

In [13]:
df_ukretail = df_ukretail[df_ukretail['Price']> 0]

Nous supprimons les lignes où le stockcode est composé uniquement de lettres pour ne pas prendre en compte tout ce qui concerne les frais de port, les tests, qui ne donnent pas de renseignements sur le client.

In [14]:
# Appliquer le filtrage sur la colonne "stock code"
df_ukretail = df_ukretail[~df_ukretail['StockCode'].str.match(r'^[A-Za-z]')]

In [15]:
df_ukretail.describe()

,Quantity,Price
count,820565.000000,820565.000000
mean,12.445481,2.967188
std,188.873328,4.452660
min,-80995.000000,0.030000
25%,2.000000,1.250000
50%,5.000000,1.950000
75%,12.000000,3.750000
max,80995.000000,649.500000


In [16]:
nan_count = df_ukretail.isna().sum()
print(nan_count.sort_values)

<bound method Series.sort_values of Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
dtype: int64>


In [17]:
df_ukretail = df_ukretail[df_ukretail['Quantity']>0]

### Feature engineering

On peut ajouter une colonne qui nous donne le prix unitaire multiplié par la quantité.

In [18]:
df_ukretail['total'] = df_ukretail['Quantity']*df_ukretail['Price']

In [19]:
df_ukretail.columns

Index(['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'Price', 'Customer ID', 'Country', 'total'],
      dtype='object')

### Group by

In [20]:
df_gp_invoice = df_ukretail.groupby(['Customer ID','Invoice']).sum().drop('Price',axis=1)
df_gp_invoice

/tmp/ipykernel_13646/2962250399.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_gp_invoice = df_ukretail.groupby(['Customer ID','Invoice']).sum().drop('Price',axis=1)


Quantity     total
Customer ID Invoice                    
12346.0     499763          5     27.05
            513774         19    142.31
            541431      74215  77183.60
12347.0     529924        509    611.53
            537626        638   1423.58
...                       ...       ...
18287.0     523290         12     30.60
            534346        186    331.50
            554065        488    765.28
            570715        990   1001.32
            573167        108     70.68

[36594 rows x 2 columns]

In [49]:
df_rfm2 = df_ukretail.groupby('Customer ID').agg({'Invoice': 'nunique','Quantity':'sum' ,'InvoiceDate': 'max', 'total':'sum'})
# Renommer les colonnes du DataFrame df_rfm
df_rfm2.columns = ['NombreCommandes', 'NombreArticles','DateDerniereCommande', 'MontantTotal']

df_rfm2 = df_rfm2[['MontantTotal','NombreArticles']]
df_rfm2

,MontantTotal,NombreArticles
Customer ID,,
12346.0,77352.96,74239
12347.0,5633.32,3286
12348.0,1658.40,2704
12349.0,3678.69,1621
12350.0,294.40,196
...,...,...
18283.0,2730.70,1731
18284.0,411.68,493
18285.0,377.00,144


In [50]:
df_rfm2.to_csv('df_rfm2.csv')